In [1]:
from process_library import *
import os 

In [2]:
rng = np.random.RandomState(42)

# data loading 
path_to_parent_folder = "/".join( os.getcwd().split("/")[:-1] ) 
path_to_data = "data/pubchem_data_2023-12-05_343_samples.csv"
path = os.path.join(path_to_parent_folder,path_to_data)

df_all_data = pd.read_csv(path)
feature_names_involved_in_training = df_all_data.drop(['substituent','molecular_formula','Name','canonical_smiles','ΔVC-m','cid','ΔVC -p'],axis=1).columns
# target_feature_name ="ΔVC-p"

In [3]:
X_train, X_val, X_test, y_train, y_val, y_test = get_data_separated(df_all_data,random_state=rng,choice=2)
X_train, X_val, X_test = scaling_data(X_train, X_val, X_test)

In [4]:
target_feature_name = "-".join(y_train.columns.to_list()).replace(" ","")

In [5]:
regression_models = load_regression_models()


The following algorithms do not accept multiple columns targets : 
["Bayesian Ridge" ,"Huber Regressor", "Passive Aggressive Regressor","ARD Regression","Gradient Boosting","Support Vector Regression","Stochastic Gradient Descent"]

This is a constrain in the selection of the regressor algorithms

In [6]:
val_results_list = []
for name, model in regression_models.items():
    print (name)
    if name not in ["Bayesian Ridge" ,"Huber Regressor", "Passive Aggressive Regressor","ARD Regression","Gradient Boosting","Support Vector Regression","Stochastic Gradient Descent"]:
        model.fit(X_train, y_train.to_numpy())
        val_results_list.append ( save_metrics_results(model=model,X_test=X_val,y_test=y_val,tag=name) ) 



Linear Regression
Ridge
Lasso
Elastic Net
Bayesian Ridge
Huber Regressor
Passive Aggressive Regressor
RANSAC Regressor
ARD Regression
Orthogonal Matching Pursuit
Decision Tree
Random Forest


/home/barradd/Documents/GitHub/machine_learning_chem_RGS/env/lib/python3.10/site-packages/sklearn/linear_model/_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), OrthogonalMatchingPursuit())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(


Gradient Boosting
K-Nearest Neighbors
Support Vector Regression
Stochastic Gradient Descent
Multi-layer Perceptron


/home/barradd/Documents/GitHub/machine_learning_chem_RGS/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [7]:
val_results_df = pd.concat(val_results_list)
val_results_df.sort_values(by=["R^2", "MAE"],ascending=False)

,R^2,MAE,MSE,MAPE,MedAE
Multi-layer Perceptron,9.254232e-01,1.548878e+00,4.875440e+00,5.022526e-01,1.046721
Ridge,9.089691e-01,1.777687e+00,6.034139e+00,7.214352e-01,1.311270
Linear Regression,9.056123e-01,1.798061e+00,6.253491e+00,7.291150e-01,1.234517
Random Forest,8.747634e-01,1.657078e+00,8.282271e+00,4.996769e-01,0.665322
Orthogonal Matching Pursuit,8.715649e-01,2.123982e+00,8.323944e+00,1.052336e+00,1.643018
K-Nearest Neighbors,8.586544e-01,2.171421e+00,9.370296e+00,6.545082e-01,1.341478
Elastic Net,8.447852e-01,2.494573e+00,1.025083e+01,1.042713e+00,1.943617
Lasso,8.358426e-01,2.623597e+00,1.073058e+01,1.150902e+00,2.023249
Decision Tree,8.252198e-01,1.732690e+00,1.143719e+01,3.276806e-01,0.284287
RANSAC Regressor,-2.193452e+15,8.482910e+07,1.504002e+17,1.623599e+07,2.058848
